In [1]:
# Installing dependencies
import numpy as np
import pandas as pd
from yfQuery import datareader

In [2]:
class DataLoader:
    '''
        Class for loading past stocks prices
        symbol: symbol can be either a single symbol or 
                a list of symbol
        start:  start date
        end:    end date
    '''
    def __init__(self, symbol, start, end):
        # Preload
        self.data = datareader(symbol, start, end)
    
    def get(self, start, end):
        # Return a period of the data
        return self.data.iloc[start: end]

In [3]:
class BuySell:
    def __init__(self, capital=None, max_share=None):
        self.original_capital = capital
        self.capital = capital
        self.max_share = max_share
        self.share = 0
        self.is_holding = False
        self.buy_at = 0
        # Array for dollar gain per trade 
        self.dollar_gain = []
        # Array for percentage gain per trade
        self.pct_gain = []
        # Array for Buy / Sell price
        self.stat = []
    
    def buy(self, price):
        # Buy Action
        self.stat.append([price, np.nan])
        self.buy_at = price
        self.is_holding = True
        # Buy Power with capital
        if self.capital is not None:
            self.share = self.capital // self.buy_at
            if self.max_share is not None and self.share > self.max_share:
                self.share = self.max_share
            self.capital -= np.round(self.buy_at * self.share, 2)
    
    def sell(self, price):
        # Sell Action
        self.calculate_gain(price)
        self.stat.append([np.nan, price])
        if self.capital is not None:
            self.capital += np.round(price * self.share)
            self.share = 0
        self.buy_at = 0
        self.is_holding = False
    
    def no_action(self):
        self.stat.append([np.nan, np.nan])
    
    def calculate_gain(self, price):
        dollar_gain = price - self.buy_at
        pct_gain = price / self.buy_at - 1
        self.dollar_gain.append(dollar_gain)
        self.pct_gain.append(pct_gain)
    
    def show_results(self):
        self.results = self.result_Series()
        print('${:.2f} gained per share after {} trades'.format(self.results['Total-Dollar-Gain'], 
                                                                self.results['Num-Trades']))
        print('and have a total gain percentage of {:.2f}%'.format(self.results['Total-Percentage-Gain'] * 100))
        if self.capital is not None:
            print('Test Ending Capital: ${:.2f} base on original capital of ${:.2f}'.format(self.capital, self.original_capital))
            print('With {:.2f}% Capital Gain'.format(self.results['Percentage-Capital-Gain'] * 100))

    def result_Series(self):
        index = ['Num-Trades', 'Total-Dollar-Gain', 'Total-Percentage-Gain', 'Percentage-Capital-Gain']
        num_trades = len(self.dollar_gain)
        total_dollar_gain = sum(self.dollar_gain)
        total_pct_gain = sum(self.pct_gain)
        array = [num_trades, total_dollar_gain, total_pct_gain]
        if self.capital is not None:
            capital_gain = self.capital / self.original_capital - 1
            array.append(capital_gain)
            return pd.Series(array, index=index)
        return pd.Series(array, index=index[:-1])
        

In [63]:
d = DataLoader('AAPL', '2018-01-01', '2019-12-31')
prices = d.data[['Close', 'Low', 'High']]

In [64]:
# MA Crossing
a, b = 3, 6
fast_ma = prices['Close'].rolling(a).mean()
slow_ma = prices['Low'].rolling(b).mean()
above = (fast_ma > slow_ma) & (prices['Low'] > slow_ma)
data = pd.concat([fast_ma, slow_ma, above], axis=1)
data.columns = ['fast_ma', 'slow_ma', 'above']
data = pd.concat([prices, data], axis=1)
data = data.dropna()

In [65]:
length = len(data)
bs = BuySell(capital=1000, max_share=1000)

for i in range(length):
    if i == 0:
        bs.no_action()
        continue
    is_above = data['above'].iloc[i - 1]
    buying_price = data['Low'].iloc[i]
    selling_pirce = data['Close'].iloc[i]
    if i + 1 == length:
        if bs.is_holding:
            bs.sell(selling_pirce)
    else:
        if not bs.is_holding:
            if is_above:
                bs.buy(buying_price)
            else:
                bs.no_action()
        else:
            if not is_above:
                bs.sell(selling_pirce)
            else:
                bs.no_action()


In [66]:
bs.show_results()

$43.67 gained per share after 53.0 trades
and have a total gain percentage of 89.17%
Test Ending Capital: $2296.33 base on original capital of $1000.00
With 129.63% Capital Gain
